# Hello Segmentation

A very basic introduction to OpenVINO segmentation model.

We use a [Semantic Segmentation Adas](https://docs.openvinotoolkit.org/2019_R1/_semantic_segmentation_adas_0001_description_semantic_segmentation_adas_0001.html) from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/).


## Imports

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from openvino.inference_engine import IECore

sys.path.append("../utils")
from notebook_utils import segmentation_map_to_image

## Load the network

In [ ]:
ie = IECore()

net = ie.read_network(
    model="model/road-segmentation-adas-0001.xml", weights="model/road-segmentation-adas-0001.bin"
)
exec_net = ie.load_network(net, "CPU")

output_layer_ir = next(iter(exec_net.outputs))
input_layer_ir = next(iter(exec_net.input_info))

## Load an Image

In [ ]:
# The segmentation network expects images in RGB format
image = cv2.cvtColor(cv2.imread("data/coco_cross.png"), cv2.COLOR_BGR2RGB)

# N,C,H,W = batch size, number of channels, height, width
N, C, H, W = net.input_info[input_layer_ir].tensor_desc.dims

# OpenCV resize expects the destination size as (width, height)
input_image = cv2.resize(image, (W, H))  
# reshape to network input shape
input_image = np.expand_dims(
    input_image.transpose(2, 0, 1), 0
)  
plt.imshow(image)

## Do Inference

In [ ]:
# Run the infernece
result = exec_net.infer(inputs={input_layer_ir: input_image})
result_ir = result[output_layer_ir]

# Prepare data for visualization
result_ir_idx = np.argmax(result_ir, axis=1)
plt.imshow(result_ir_idx[0])


## Prepare data for visualisation

In [ ]:
# Define colormap, each color represents other object type 
colormap = np.array([[255, 0, 122], [0, 255, 0], [255, 0, 0], [255, 255, 0]])

# Define how transparent will be mask on photo
alpha = 0.15

# Use function from notebook_utils.py to create mask
mask = segmentation_map_to_image(result_ir_idx, colormap)

# Transpose image back to HWC(height, width, channels) layout
input_image_t = np.squeeze(input_image).transpose(1, 2, 0)

# Create image with mask put on
image_with_mask = cv2.addWeighted(mask, alpha, input_image_t, 1 - alpha, 0)


## Visualise data

In [ ]:
# Define titles with images
data = {'Segmentation': mask, "Base Photo": input_image_t, "Masked Photo": image_with_mask}

# Create subplot to visualise images
f, axs = plt.subplots(1, len(data.items()))

# Fill subplot
for ax, (name, image) in zip(axs, data.items()):
    ax.axis('off')
    ax.set_title(name)
    ax.imshow(image)

# Display image
plt.show(f)